<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.981 · TFM · Aula 1</p>
<p style="margin: 0; text-align:right;">2024-2 · Màster universitari en Ciència de dades (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudis d'informàtica, multimèdia i telecomunicació</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>
<div style="text-align: center; margin-top: 40px;">
    <h1>Aplicación para ayuda a la conducción con deep learning</h1>
</div>

### ARCHIVO 11: Entrenamiento semáforos

Entrena un modelo clasificador.

Código que ejecuta el entrenamiento a partir de imágenes en carpetas definidas. Modelo a entrenar: EfficientNet B0.

Código ejecutado en plataforma Visual Studio Code.

In [1]:
# Llibreries

import os
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch.nn.functional as F

In [ ]:
# Dades

Dades = "Semafors"  # Carpeta amb subcarpetes per classe

# Bloc per a la normalització d'imatges
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [3]:
# Classe per a crear un dataset

class TrafficLightDataset(Dataset):
    def __init__(self, paths, labels):
        self.paths = paths
        self.labels = labels

    def __len__(self): return len(self.paths)

    def __getitem__(self, i):
        img = cv2.imread(self.paths[i])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = transform(img)
        return img, torch.tensor(self.labels[i], dtype=torch.long)

In [ ]:
# Llegeix carpetes

# Funció per a recopilar les imatges i torna datasets separats
def get_data():
    paths, labels = [], []
    classes = sorted(os.listdir(Dades))  # Assumim subcarpetes amb nom de la classe
    class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}
    for cls in classes:
        full_folder = os.path.join(Dades, cls)
        for f in os.listdir(full_folder):
            if f.lower().endswith(('jpg', 'png', 'jpeg')):
                paths.append(os.path.join(full_folder, f))
                labels.append(class_to_idx[cls])
    return train_test_split(paths, labels, test_size=0.2, random_state=42), class_to_idx

# Dades i loaders
(train_paths, test_paths, train_labels, test_labels), class_to_idx = get_data()
train_loader = DataLoader(TrafficLightDataset(train_paths, train_labels), batch_size=32, shuffle=True)
test_loader = DataLoader(TrafficLightDataset(test_paths, test_labels), batch_size=32)

In [5]:
# Càrrega model

num_classes = len(class_to_idx)
model = models.efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.cuda()

# Paràmetres
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

c:\Users\malve\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\malve\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# Entrenament

model.train()
for epoch in range(15):
    total_loss = 0
    for x, y in train_loader:
        x, y = x.cuda(), y.cuda()
        out = model(x)
        loss = loss_fn(out, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: {total_loss/len(train_loader):.4f}")

torch.save(model, "TrafficLight2.pt")

# Avaluació
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for x, y in test_loader:
        x = x.cuda()
        out = model(x)
        preds = out.argmax(dim=1).cpu()
        y_true.extend(y)
        y_pred.extend(preds)

# Mètriques
print("Mètriques de validació:")
print("Accuracy :", accuracy_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred, average='weighted'))
print("Recall   :", recall_score(y_true, y_pred, average='weighted'))
print("F1 Score :", f1_score(y_true, y_pred, average='weighted'))

print("Fi d'entrenament")

Epoch 1: 1.5042
Epoch 2: 1.1760
Epoch 3: 0.9029
Epoch 4: 0.7389
Epoch 5: 0.5356
Epoch 6: 0.4127
Epoch 7: 0.3018
Epoch 8: 0.2848
Epoch 9: 0.2677
Epoch 10: 0.1799
Epoch 11: 0.1390
Epoch 12: 0.1488
Epoch 13: 0.1779
Epoch 14: 0.1335
Epoch 15: 0.1370
Mètriques de validació:
Accuracy : 0.9390243902439024
Precision: 0.9441980783444198
Recall   : 0.9390243902439024
F1 Score : 0.9334221493475439
Fi d'entrenament
